In [1]:
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import itertools
from functools import partial

%matplotlib inline

### data loader

In [2]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.RandomCrop(200)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=25)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

In [3]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.RandomCrop(200)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/valid/', transform=transforms)
validloader = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=False, num_workers=4)

### resnet-34

In [7]:
model = torchvision.models.resnet34(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(512, 12)

criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

### optimal learning rate

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
learning_rates = [10**x/100000000 for x in range(0,10)]

In [9]:
losses = []

for learning_rate in learning_rates:
    # assign learning rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    # read data
    for i in range(6):
        _loss = 0.0 
        inputs, labels = next(iter(trainloader))
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs = Variable(inputs)
            labels = Variable(labels)

        # calculate loss and output
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _loss += loss.data[0]
    losses.append(_loss)
    
plt.figure(figsize=(10,5))
plt.plot(losses)
plt.xticks(range(len(losses)), learning_rates[:3])
plt.title('LR vs Loss')
plt.show()

KeyboardInterrupt: 

### tune model

In [10]:
model = torchvision.models.resnet34(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(512, 12)

criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [12]:
def cosine_anneal(min_val, max_val, total_steps, step):
    lr = min_val
    cosine = np.cos(np.pi * step / total_steps)
    lr += 0.5*(max_val - min_val)*(1 + cosine)
    return lr

cosine_anneal = partial(cosine_anneal, 0.001, 0.006)

In [13]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
validloader = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=False, num_workers=4)

In [14]:
def score(model, dataloader):
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.
    for data in dataloader: 
        print('hi')
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs = Variable(inputs)
            labels = Variable(labels)
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        calc_loss += loss.data[0]
        calc_correct += torch.sum(preds == labels.data)
        calc_count += outputs.data.shape[0]
        break
    return calc_loss, calc_correct / calc_count, calc_count

In [ ]:
def train(trainloader, validloader, epochs, interval):
    evalloaders = itertools.tee(trainloader, epochs//interval)
    validloaders = itertools.tee(validloader, epochs//interval)
    counter = 0
    for epoch in range(10):
        # read data
        loss = 10
        for data in trainloader: 
            inputs, labels = data
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs = Variable(inputs)
                labels = Variable(labels)
            # calculate loss and output
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break
        if epoch % interval == 0:
            if counter < len(evalloaders):
                print('training   loss:', score(model, evalloaders[counter]))
                print('validation loss:', score(model, validloaders[counter]))
                counter += 1
        return model

In [ ]:
model = train(trainloader, validloader, 1, 1)